----
# Data Cleaning
----

### Notebook Summary

In this notebook, I will prepare the date-indexed microsoft data for forecasting. This process includes:

- **Missing Dates:** Identify any missing dates in the dataset. If necessary, re-index the data to include all dates within the specified 5-year period.
- **Missing Values:** Idendify and address any missing values in the dataset.
- **Exporting Cleaned Data:** Export the processed dataframe to CSV ready for EDA.

## Set Up
---

In [26]:
import numpy as np
import pandas as pd

import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objs as go
import matplotlib.pyplot as plt
import seaborn as sns

from statsmodels.api import tsa # time series analysis
import statsmodels.api as sm

## Data Loading
----

In [27]:
# loading data from csv, set index to be first column (our date column from 01-data-loading)
msft_df = pd.read_csv('../../data/msft_data.csv', index_col=0)

In [28]:
msft_df.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits
2019-09-12,131.667405,132.211832,130.731347,131.352203,27010000,0.0,0.0
2019-09-13,131.600570,131.868011,130.444847,131.161209,23363100,0.0,0.0
2019-09-16,129.738001,130.568977,129.575628,130.215576,16731400,0.0,0.0
2019-09-17,130.817320,131.352201,130.311077,131.228027,17814200,0.0,0.0
2019-09-18,131.199380,132.450624,130.406604,132.307358,23982100,0.0,0.0


## Checking datatypes
---

In [29]:
msft_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1258 entries, 2019-09-12 to 2024-09-11
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1258 non-null   float64
 1   High          1258 non-null   float64
 2   Low           1258 non-null   float64
 3   Close         1258 non-null   float64
 4   Volume        1258 non-null   int64  
 5   Dividends     1258 non-null   float64
 6   Stock Splits  1258 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.6+ KB


### Reset Date to be datetime type

In [30]:
msft_df.index = pd.to_datetime(msft_df.index)

In [31]:
msft_df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1258 entries, 2019-09-12 to 2024-09-11
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Open          1258 non-null   float64
 1   High          1258 non-null   float64
 2   Low           1258 non-null   float64
 3   Close         1258 non-null   float64
 4   Volume        1258 non-null   int64  
 5   Dividends     1258 non-null   float64
 6   Stock Splits  1258 non-null   float64
dtypes: float64(6), int64(1)
memory usage: 78.6 KB


**Comment:**
Remaining datatypes are numerical and so we can continue with data cleaning

## Looking for missing dates
---

In [32]:
# Getting first and last day from dataset
first_day = msft_df.index.min()
last_day = msft_df.index.max()

In [33]:
# Calculate difference between last and first day
last_day -  first_day

Timedelta('1826 days 00:00:00')

In [34]:
msft_df.shape

(1258, 7)

**Comment:**

There is a difference of 568 days, I assune these dates are non-trading days such as weekends and US bank holidays. These dates will need to be filled in order to perform timeseries forecasting since the methods to be used require a continour date range.

See Appendix/Missing Dates for checking dates are non-trading days (weekends/bank holidays).


## Checking for missing values
----

In [35]:
msft_df.isna().sum() 

Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

**Comment:** 

As expected, there are no missing values in the dataset. Yahoo Finance provides a clean dataset with minimal need for data cleaning.

## Reindexing dates
---

Before filling the missing non-trading dates, I will need to reindex the  datarange to ensure the index (dates) are continous.

In [50]:
# Assuming df is your DataFrame with a DateTime index
# Reindex to include all days (daily frequency)
full_index = pd.date_range(start=first_day, end=last_day, freq='D')


In [51]:
msft_df = msft_df.reindex(full_index)

### Checking date range is now continuous

In [52]:
full_range = pd.date_range(start=first_day, end=last_day, freq='D')

In [53]:
full_range.difference(msft_df.index)

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

No longer missing dates in date range, to continue in analysis.

In [54]:
msft_df.isna().sum()

Open            569
High            569
Low             569
Close           569
Volume          569
Dividends       569
Stock Splits    569
dtype: int64

Now there are missing values for the non-trading dates added during re-indexing.

### Filling values for missing dates using Interpolation


Interpolation is a method that estimates missing data points by assuming a linear relationship between the surrounding values. In other words, it looks at the numbers and before the null values and assumes the change between the two is smooth.

In [55]:
# Linear interpolation
msft_df= msft_df.interpolate(method='linear')

In [56]:
msft_df.isna().sum()

Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock Splits    0
dtype: int64

No longer see any null values, to export the cleaned dataset for EDA.

## Exporting clean data
----

In [57]:
msft_df.to_csv('../../data/msft_cleaned.csv')

## Conclusion
---

After filling in the missing dates and values, the dataset is now clean and ready for further analysis. 

I will move on to Exploratory Data Analysis (EDA), where we can begin uncovering insights and patterns from the data.

## Appendix
-----

### Missing Dates

In [36]:
# Calculate full date range between first and last day
# Using freq B to ensure dates reflect business days (excludes weekends and BHs)
full_range = pd.date_range(start=first_day, end=last_day, freq="B")

In [37]:
difference = full_range.difference(msft_df.index)

In [38]:
difference.shape

(47,)

**Comment:** 

There are 47 missing days over the 5-year period. I will review these missing dates to determine if there is an underlying reason as to they are missing. Given that Yahoo Finance typically provides a clean dataset, I expect there may be specific reasons for these missing days, especially after considering business days only.

In [39]:
difference

DatetimeIndex(['2019-11-28', '2019-12-25', '2020-01-01', '2020-01-20',
               '2020-02-17', '2020-04-10', '2020-05-25', '2020-07-03',
               '2020-09-07', '2020-11-26', '2020-12-25', '2021-01-01',
               '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31',
               '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24',
               '2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30',
               '2022-06-20', '2022-07-04', '2022-09-05', '2022-11-24',
               '2022-12-26', '2023-01-02', '2023-01-16', '2023-02-20',
               '2023-04-07', '2023-05-29', '2023-06-19', '2023-07-04',
               '2023-09-04', '2023-11-23', '2023-12-25', '2024-01-01',
               '2024-01-15', '2024-02-19', '2024-03-29', '2024-05-27',
               '2024-06-19', '2024-07-04', '2024-09-02'],
              dtype='datetime64[ns]', freq=None)

**Comment:**

On investigating the missing dates, it looks like they correspond to US bank holidays. Since the stock market is closed on these dates, we should exclude them in addition to weekends when analysing the dataset. To do this, I will use `holidays` library and `CustomBusinessDay` module to include US bank holidays in the list of dates to exclude.

In [40]:
from pandas.tseries.offsets import CustomBusinessDay
import holidays

In [41]:
us_bank_hols = holidays.UnitedStates(years=[2019,2020,2021,2022,2023,2024])

In [42]:
# Now excluding weekends AND US bank holidays
cust_business_days = CustomBusinessDay(holidays=us_bank_hols)

In [43]:
# Recalcualting full date range between first and last day
# Now excluding weekends and US holidays 
business_days = pd.date_range(start=first_day, end=last_day, freq=cust_business_days)

In [44]:
business_days.difference(msft_df.index)

DatetimeIndex(['2020-04-10', '2021-04-02', '2022-04-15', '2023-04-07',
               '2024-03-29'],
              dtype='datetime64[ns]', freq=None)

**Comment:**

Only 5 dates missing now, one per year which is intereseting.

After looking into these dates futher, it appears these dates represent Good Friday holiday for each year. For some reason these holidays were not exlcuded in us_bank_hols: 

    April 10, 2020: Good Friday
    April 2, 2021: Good Friday
    April 15, 2022: Good Friday
    April 7, 2023: Good Friday
    March 29, 2024: Good Friday



In [45]:
# Define Good Friday dates (for 2019 to 2024)
good_fridays = pd.to_datetime([
    '2019-04-19',  # Good Friday 2019
    '2020-04-10',  # Good Friday 2020
    '2021-04-02',  # Good Friday 2021
    '2022-04-15',  # Good Friday 2022
    '2023-04-07',  # Good Friday 2023
    '2024-03-29',  # Good Friday 2024
])

all_holidays = pd.to_datetime(list(us_bank_hols) + list(good_fridays))
cust_business_days = CustomBusinessDay(holidays=all_holidays)

In [46]:
all_holidays = pd.to_datetime(list(us_bank_hols) + list(good_fridays))

In [47]:
# Create a custom business day calendar including these holidays
cust_business_days = CustomBusinessDay(holidays=all_holidays)

In [48]:
business_days = pd.date_range(start=first_day, end=last_day, freq=cust_business_days)

In [49]:
business_days.difference(msft_df.index)

DatetimeIndex([], dtype='datetime64[ns]', freq=None)

Confirms missing dates are all either the weekends (not business days) or US bank holidays. Both of which are non-trading days